## SAM Sequence Nuclei Detection, Quantification and Alignment

In [ ]:
import logging
import argparse
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import sam_spaghetti
from sam_spaghetti.sam_microscopy_loading import load_image_from_microscopy
from sam_spaghetti.sam_sequence_info import get_experiment_name, get_experiment_microscopy, get_nomenclature_name, get_experiment_channels, get_experiment_reference, get_sequence_orientation, get_experiment_microscope_orientation, update_lut_ranges
from sam_spaghetti.detection_quantification import detect_and_quantify
from sam_spaghetti.sam_sequence_loading import load_sequence_signal_images, load_sequence_signal_image_slices, load_sequence_signal_data
from sam_spaghetti.signal_image_slices import sequence_signal_image_slices, sequence_image_primordium_slices, sequence_signal_data_primordium_slices
from sam_spaghetti.signal_image_plot import signal_image_plot, signal_nuclei_plot, signal_map_plot, signal_image_all_primordia_plot, signal_nuclei_all_primordia_plot, signal_map_all_primordia_plot
from sam_spaghetti.signal_map_computation import compute_signal_maps, compute_primordia_signal_maps, compute_average_signal_maps, compute_average_primordia_signal_maps
from sam_spaghetti.sequence_image_registration import register_sequence_images, apply_sequence_registration
from sam_spaghetti.signal_data_compilation import compile_signal_data, compile_primordia_data
from sam_spaghetti.sequence_growth_estimation import compute_growth
from sam_spaghetti.sam_sequence_primordia_alignment import align_sam_sequence, detect_organ_primordia
from sam_spaghetti.data_visualization import primordium_data_plot

logging.getLogger().setLevel(logging.ERROR)

%matplotlib inline

### Parameters

In [ ]:
data_dirname = sam_spaghetti.__path__[0]+"/../../share/data"
microscopy_dirname = data_dirname+"/microscopy"
image_dirname = data_dirname+"/nuclei_images"

In [ ]:
experiment = 'E35'
sam_id = 4

detection = False
registration = False
primordia_alignment = True

projection_type = "max_intensity"
signals_to_display = ['CLV3','DIIV']

### Microscopy image loading & Nuclei detection

In [ ]:
experiment_name = get_experiment_name(experiment,data_dirname)
if experiment_name == "":
    logging.error("Experiment identifier \""+experiment+"\" not recognized (consider adding it to the experiment_info.csv file in the data directory)")
else:
    if detection and (microscopy_dirname is not None):
        experiment_dirname = microscopy_dirname+"/"+get_experiment_microscopy(experiment,data_dirname)
        if os.path.exists(experiment_dirname+"/RAW"):
            experiment_dirname += "/RAW"

        if not os.path.exists(experiment_dirname):
            logging.warning("Microscopy directory not found for "+experiment+", no detection will be performed.")
        else:                    
            microscopy_filenames = [experiment_dirname+"/"+f for f in os.listdir(experiment_dirname) if np.any([ext in f for ext in ['.czi','.lsm','.tif']])]
            nomenclature_names = [get_nomenclature_name(microscopy_filename,data_dirname) for microscopy_filename in microscopy_filenames]
            microscopy_filenames = [m for m,n in zip(microscopy_filenames,nomenclature_names) if n is not None and "_sam"+str(sam_id).zfill(2) in n]
            
            channel_names = get_experiment_channels(experiment, data_dirname)
            reference_name = get_experiment_reference(experiment, data_dirname)
        
            for microscopy_filename in microscopy_filenames:
                nomenclature_name = get_nomenclature_name(microscopy_filename,data_dirname)

                if nomenclature_name is not None:
                    sequence_name = nomenclature_name[:-4]
                    if not os.path.exists(image_dirname):
                        os.makedirs(image_dirname)
                    if not os.path.exists(image_dirname+"/"+sequence_name):
                        os.makedirs(image_dirname+"/"+sequence_name)
                    if not os.path.exists(image_dirname+"/"+sequence_name+"/"+nomenclature_name):
                        os.makedirs(image_dirname+"/"+sequence_name+"/"+nomenclature_name)

                    if detection:
                        logging.info("--> Running detection on "+nomenclature_name+" "+reference_name)
                        img_dict = load_image_from_microscopy(microscopy_filename, save_images=True, image_dirname=image_dirname, nomenclature_name=nomenclature_name, channel_names=channel_names, verbose=False, loglevel=1)
                        detect_and_quantify(img_dict, reference_name=reference_name, signal_names=channel_names, image_dirname=image_dirname, nomenclature_name=nomenclature_name, verbose=False, loglevel=1)
                else:
                    logging.warning("--> No nomenclature found for " + microscopy_filename + ", skipping...")

### Multichannel image sequence

In [ ]:
sequence_name = experiment_name + "_sam" + str(sam_id).zfill(2)
reference_name = get_experiment_reference(experiment, data_dirname)

logging.info("--> Loading sequence " + sequence_name)

signal_images = load_sequence_signal_images(sequence_name, image_dirname, signal_names=[reference_name]+signals_to_display, verbose=False, loglevel=1)

if len(signal_images)==0:
    logging.error("--> Unable to load sequence " + sequence_name + " (make sure you have run detection first)")
    
microscope_orientation = get_experiment_microscope_orientation(experiment, data_dirname)

logging.info("--> Plotting signal images " + sequence_name)
signal_image_slices = load_sequence_signal_image_slices(sequence_name, image_dirname, projection_type=projection_type, aligned=False, verbose=False, loglevel=1)
if len(signal_image_slices) == 0:
    signal_image_slices = sequence_signal_image_slices(sequence_name, image_dirname, reference_name=reference_name, microscope_orientation=microscope_orientation,
                                                       projection_type=projection_type, resolution=None, aligned=False, save_files=True, verbose=False, loglevel=1)

figure = signal_image_plot({s:signal_image_slices[s] for s in [reference_name]+signals_to_display}, reference_name=reference_name, projection_type=projection_type, resolution=0.25, aligned=False, verbose=False, loglevel=1)

### L1 Nuclei signal quantification

In [ ]:
r_max = list(signal_images[reference_name].values())[0].shape[0] * list(signal_images[reference_name].values())[0].voxelsize[0] / 2.
signal_data = load_sequence_signal_data(sequence_name, image_dirname, normalized=False, aligned=False, verbose=False, loglevel=1)

logging.info("--> Plotting detected nuclei signals " + sequence_name)
figure = signal_nuclei_plot(signal_data, r_max=r_max, signal_names=signals_to_display, normalized=False, verbose=False, loglevel=1)


### 2D Maps of epidermal Auxin level

In [ ]:
logging.info("--> Compiling signal data")
compile_signal_data([experiment], save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, verbose=False, loglevel=1)
normalized_signal_data = load_sequence_signal_data(sequence_name, image_dirname, normalized=True, aligned=False, verbose=False, loglevel=1)

figure = signal_nuclei_plot(normalized_signal_data, r_max=r_max, signal_names=['Auxin'], normalized=False, verbose=False, loglevel=1)

In [ ]:
signal_maps = compute_signal_maps(normalized_signal_data, signal_names=['Auxin'], normalized=False, polar=False, verbose=False, loglevel=1)
logging.info("--> Plotting maps "+sequence_name)
figure = signal_map_plot(signal_maps, verbose=False, loglevel=1)

### Sequence rigid image registration

In [ ]:
if registration:
    logging.info("--> Sequence image registration "+sequence_name)
    register_sequence_images(sequence_name, microscope_orientation=microscope_orientation, pyramid_lowest_level=3, save_files=True, image_dirname=image_dirname, reference_name=reference_name, verbose=False, loglevel=1)
apply_sequence_registration(sequence_name, microscope_orientation=microscope_orientation, save_files=True, image_dirname=image_dirname, reference_name=reference_name, verbose=False, loglevel=1)

registered_signal_image_slices = sequence_signal_image_slices(sequence_name, image_dirname, reference_name=reference_name, microscope_orientation=microscope_orientation, projection_type=projection_type, resolution=None, registered=True, aligned=False, save_files=False, verbose=False, loglevel=1)
figure = signal_image_plot({reference_name:registered_signal_image_slices[reference_name]}, reference_name=reference_name, projection_type=projection_type, resolution=None, aligned=False, verbose=False, loglevel=1)

### Sequence alignment : CZ centering + P0 identification

In [ ]:
logging.info("--> Compiling signal data")
compile_signal_data([experiment], save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, verbose=False, loglevel=1)

if primordia_alignment:
    logging.info("--> Sequence primordia alignment "+sequence_name)
    sam_orientation = get_sequence_orientation(sequence_name,data_dirname)
    sequence_aligned_data = align_sam_sequence(sequence_name, image_dirname, sam_orientation=sam_orientation, save_files=True, verbose=False, loglevel=1)

### Aligned signal maps

In [ ]:
aligned_signal_data = load_sequence_signal_data(sequence_name, image_dirname, normalized=True, aligned=True, verbose=False, loglevel=1)

aligned_signal_maps = compute_signal_maps(aligned_signal_data, signal_names=['CLV3','Auxin'], normalized=False, aligned=True, polar=False, verbose=False, loglevel=1)
logging.info("--> Plotting aligned maps "+sequence_name)
figure = signal_map_plot(aligned_signal_maps, aligned=True, verbose=False, loglevel=1)

### Detection of organ primordia

In [ ]:
detect_organ_primordia(sequence_name, image_dirname,  cell_radius=5, density_k=0.25, save_files=True, verbose=False, loglevel=1)

In [ ]:
logging.info("--> Compiling signal data and primordia data")
compile_signal_data([experiment],save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, aligned=True, verbose=False, loglevel=1)
sequence_primordia_data = compile_primordia_data([experiment],save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, verbose=False, loglevel=1)

#primordium_range = [0]
primordium_range = [-3,-2,-1,0,1,2,3,4,5]
time_range = [0,5,10]

for i_s,signal_name in enumerate(['radial_distance','Auxin']):
    figure = plt.figure(i_s)
    figure.clf()
    primordium_data_plot(sequence_primordia_data,figure,signal_name=signal_name,primordium_range=primordium_range,time_range=time_range,data_to_plot='detected_auxin_extrema',figure_size=(18,6))

